## 5.1 建立信用评估指标体系

在上一部分中，我们对数据进行了详细的预处理，接下来我们则需要通过建立指标体系来对不同的字段进行分类，这也为下一步的建模工作做准备。参照国际信用评分指标体系的建立方式，我们开始建立信用评估指标体系。指标体系分为个人信息、信用历史、偿债能力、消费能力四大指标类。为了更详细的刻画客户的信用信息，还可以根据已有指标来建立一些新的指标，例如“不良记录”、“平均每笔取现金额“等等。

![](https://ai-studio-static-online.cdn.bcebos.com/db90029e7677409182f923d961692e8e2f1ae3c7d7ac4529a95c17314cf7e970)


新增的指标包括：

构造客户的年消费总额，对该客户的消费能力进行初步判断：

                            年消费总额=年消费笔数均值×年消费金额均值

构造客户的年取现总额，从侧面评估该客户的总资产：

                            年取现总额=年取现笔数均值×年取现金额均值

构造客户平均每笔取现金额，评估客户在资产方面是否具有稳定性：

                            平均每笔取现金额=总取现金额/总取现笔数

构造客户网上平均每笔消费金额，评估客户的网络消费习惯：
 
                            网上平均每笔消费额=网上消费金额/网上消费笔数

消费与收入息息相关，构造公共事业平均每笔缴费额，可以更好地衡量客户的消费支出情况：

                            公共事业平均每笔缴费额=公共事业缴费金额/公共事业缴费笔数

构造客户的不良记录评分，不良记录越多，分数越高，可以更好地评估客户信用风险：

                            不良记录=法院失信传唤记录+有无逾期记录+有无犯罪记录+黑名单接口记录不良记录



## 5.2 年消费总额指标计算 

我们可以通过整合一些已有数据来帮助我们进一步的分析。

首先我们可以通过查看客户的年消费总额来对该客户的消费能力进行初步判断，已知客户年消费笔数均值及年消费金额均值，因此：

                         年消费总额 = transCnt_mean × transAmt_mean


## 实训任务

- 请用上述公式计算出客户的年消费总额，将结果赋值给trans_total。
- 将计算结果保留到小数点后六位，将结果赋值给trans_total。
- 将结果加在data数据集中的最后一列，并将此列命名为trans_total。

In [1]:
import pandas as pd
import pandas as pd
data = pd.read_table('dataset5.txt',sep='\t')
# 计算客户年消费总额。
trans_total = data['transCnt_mean'] * data['transAmt_mean']

# 将计算结果保留到小数点后六位。
trans_total = round(trans_total, 6)

# 将结果加在data数据集中的最后一列，并将此列命名为trans_total。
data['trans_total'] = trans_total

print(data['trans_total'].head(20))

0          0.000000
1        180.000000
2        792.962962
3      42012.000000
4       1515.600000
5         37.500000
6       8503.703705
7          0.000000
8     928364.444143
9       2697.777779
10       446.913580
11         0.000000
12       425.061728
13     14499.000000
14      1238.888889
15      4000.000000
16       445.000000
17        36.111111
18        90.000000
19         0.000000
Name: trans_total, dtype: float64


## 5.3 年取现总额指标计算

我们还可以通过判断计算客户的年取现总额来简单评估该客户的总资产，我们已知客户年取现笔数均值及年取现金额均值，因此： 

                         年取现总额 = cashCnt_mean × cashAmt_mean

## 实训任务

- 请用上述公式计算出客户年取现总额。
- 将计算结果保留到小数点后六位。
- 将结果加在data数据集的最后一列，并将此列命名为total_withdraw。

In [3]:
import pandas as pd
import pandas as pd
data = pd.read_table('dataset6.txt',sep='\t')
# 年取现总额 = cashCnt_mean × cashAmt_mean，计算客户年取现总额。
total_withdraw = data['cashCnt_mean'] * data['cashAmt_mean']
# 将计算结果保留到小数点后六位。
total_withdraw = round(total_withdraw, 6)
# 将结果加在data数据集的最后一列，并将此列命名为total_withdraw。
data['total_withdraw'] = total_withdraw

print(data['total_withdraw'].head(20))


0          0.000000
1          0.000000
2          0.000000
3     132000.000000
4          0.000000
5      10500.000000
6       5760.000000
7          0.000000
8       2555.555556
9          0.000000
10      1500.000000
11         0.000000
12      1244.444444
13      8166.666666
14       100.000000
15         0.000000
16     13701.562500
17       900.000000
18      1200.000000
19         0.000000
Name: total_withdraw, dtype: float64


## 5.4 平均每笔取现金额指标计算

已知客户的总取现金额和总取现笔数，我们可以构造平均每笔取现金额，推断客户资产方面是否具有稳定性。

                      平均每笔取现金额 = cashTotalAmt/cashTotalCnt
在这一步中，可能出现总取现金额和总取现笔数为0的情况，直接进行0除0的运算会导致结果变为NaN，直接进行一个数除0的运算会导致结果变为inf(无穷)，我们要考虑对这些异常情况进行处理。

## 实训任务

- 请用上述公式计算出客户的平均每笔取现金额。
- 对inf和NaN等值用0进行替换。
- 将计算结果保留到小数点后六位。
- 将结果加在data数据集的最后一列，并将此列命名为avg_per_withdraw。


In [5]:
import numpy as np
import pandas as pd
data = pd.read_table('dataset7.txt',sep='\t')
# 平均每笔取现金额 = cashTotalAmt/cashTotalCnt，计算客户的平均每笔取现金额。
avg_per_withdraw = data['cashTotalAmt'] / data['cashTotalCnt']
# 将所有的inf和NaN变为0。
avg_per_withdraw = np.nan_to_num(avg_per_withdraw, nan=0, posinf=0, neginf=0)
# 将计算结果保留到小数点后六位。
avg_per_withdraw = np.round(avg_per_withdraw, 6)
# 将结果加在data数据集的最后一列，并将此列命名为avg_per_withdraw。
data['avg_per_withdraw'] = avg_per_withdraw

print(data['avg_per_withdraw'].head(20))

0        0.000000
1        0.000000
2        0.000000
3     3666.666667
4        0.000000
5      420.000000
6      562.500000
7        0.000000
8      920.000000
9        0.000000
10     375.000000
11       0.000000
12     700.000000
13    1500.000000
14     100.000000
15       0.000000
16     640.540541
17     900.000000
18    1200.000000
19       0.000000
Name: avg_per_withdraw, dtype: float64


## 5.5 网上平均每笔消费额指标计算

为了进一步了解客户的消费习惯，我们还可以计算客户网上平均每笔消费额，然后与总消费金额进行比较。我们已知总的网上消费金额和网上消费笔数，因此只需：

                          网上平均每笔消费额 = onlineTransAmt/onlineTransCnt
在这一步中，可能出现网上消费金额和网上消费笔数为0的情况，直接进行0除0的运算会导致结果变为NaN，直接进行一个数除0的运算会导致结果变为inf(无穷)，我们要考虑对这些异常情况进行处理。


## 实训任务

- 请用上述公式计算出客户的网上平均每笔消费额。
- 对inf和NaN等值用0进行替换。
- 将计算结果保留到小数点后六位。
- 将结果加在data数据集的最后一列，并将此列命名为avg_per_online_spend。


In [7]:
import numpy as np
import pandas as pd
data = pd.read_table('dataset8.txt',sep='\t')
# 网上平均每笔消费额 = onlineTransAmt/onlineTransCnt，请计算客户的网上平均每笔消费额。
avg_per_online_spend = data['onlineTransAmt'] / data['onlineTransCnt']

# 将所有的inf和NaN变为0。
avg_per_online_spend = np.nan_to_num(avg_per_online_spend, nan=0, posinf=0, neginf=0)

# 将计算结果保留到小数点后六位。
avg_per_online_spend = np.round(avg_per_online_spend, 6)

# 将结果加在data数据集的最后一列，并将此列命名为avg_per_online_spend。
data['avg_per_online_spend'] = avg_per_online_spend

print(data['avg_per_online_spend'].head(20))

0    -3855.000000
1      233.333333
2    -4219.487179
3     1808.571429
4     -350.666667
5      -14.166667
6      985.714286
7        0.000000
8     -757.123288
9    -3058.333333
10     -30.000000
11       0.000000
12       0.000000
13    8284.444444
14       0.000000
15    4000.000000
16       0.000000
17       0.000000
18       0.000000
19       0.000000
Name: avg_per_online_spend, dtype: float64


## 5.6 公共事业平均每笔缴费额指标计算

客户的公共事业缴费金额也与客户的收入及支出息息相关，因此我们可以计算公共事业平均每笔缴费额。已知公共事业缴费金额及公共事业缴费笔数，因此只需：

                           公共事业平均每笔缴费额 = publicPayAmt/publicPayCnt
在这一步中，可能出现公共事业缴费金额和公共事业缴费笔数为0的情况，直接进行0除0的运算会导致结果变为NaN，直接进行一个数除0的运算会导致结果变为inf(无穷)，我们要考虑对这些异常情况进行处理。


## 实训任务

- 请用上述公式计算出客户的公共事业平均每笔缴费额。
- 对inf和NaN等值用0进行替换。
- 将计算结果保留到小数点后六位。
- 将结果加在data数据集的最后一列，并将此列命名为avg_per_public_spend。


In [9]:
import numpy as np
import pandas as pd
data = pd.read_table('dataset9.txt',sep='\t')
# 公共事业平均每笔缴费额 = publicPayAmt/publicPayCnt，请计算客户的公共事业平均每笔缴费额。
avg_per_public_spend = data['publicPayAmt'] / data['publicPayCnt']
# 将所有的inf和NaN变为0。
avg_per_public_spend = np.nan_to_num(avg_per_public_spend, nan=0, posinf=0, neginf=0)
# 将计算结果保留到小数点后六位。
avg_per_public_spend = np.round(avg_per_public_spend, 6)
# 将结果加在data数据集的最后一列，并将此列命名为avg_per_public_spend。
data['avg_per_public_spend'] = avg_per_public_spend

print(data['avg_per_public_spend'].head(20))

0         0.000000
1         0.000000
2       385.000000
3         0.000000
4       521.250000
5        85.714286
6      1656.666667
7         0.000000
8     -8944.285714
9     11410.000000
10      195.000000
11        0.000000
12        0.000000
13        0.000000
14        0.000000
15        0.000000
16        0.000000
17        0.000000
18      210.000000
19        0.000000
Name: avg_per_public_spend, dtype: float64


## 5.7 不良记录指标计算

在数据集中，法院失信传唤记录、有无逾期记录、有无犯罪记录及黑名单信息查询记录属于不良记录，由于这四个字段均为离散型字段，即：0：没有，1：有，因此我们可以将这四者相加，得数越多的客户即为不良记录分数越高，以此可以更方便的估计客户的信用风险。 

                          不良记录 = inCourt + isDue + isCrime + isBlackList


## 实训任务

- 请用上述公式计算出客户的不良记录分数。
- 将计算结果加在data数据集的最后一列，并将此列命名为bad_record。


In [11]:

import pandas as pd
data = pd.read_table('dataset10.txt',sep='\t')
#不良记录 = inCourt + isDue + isCrime + isBlackList请计算客户的不良记录分数。
bad_record = data['inCourt'] + data['isDue'] + data['isCrime'] + data['isBlackList']
#将计算结果加在data数据集的最后一列，并将此列命名为bad_record。
data['bad_record'] = bad_record

print(data['bad_record'].head(20))

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     1
8     0
9     0
10    0
11    0
12    1
13    0
14    0
15    0
16    0
17    0
18    0
19    0
Name: bad_record, dtype: int64
